In [1]:
import pandas as pd
import numpy as np
# import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler , Normalizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.stats import norm
from scipy import stats
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [47]:
# COVID-19 
# df = pd.read_csv("Cleaned-Data.csv")

In [104]:
# Cardio 
df = pd.read_csv("cardio_train.csv",sep=";")

In [105]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [170]:
df["age"] = df["age"]/365

In [171]:
df["age"]

0        50.391781
1        55.419178
2        51.663014
3        48.282192
4        47.873973
           ...    
69995    52.712329
69996    61.920548
69997    52.235616
69998    61.454795
69999    56.273973
Name: age, Length: 70000, dtype: float64

In [172]:
df.shape

(70000, 13)

In [173]:
df.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,53.339358,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,6.759594,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,29.583562,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,48.394521,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,53.980822,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,58.430137,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,64.967123,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [44]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

LogisticRegression(max_iter=200)

In [176]:
X_train.shape

(52500, 11)

In [46]:
# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.520


In [107]:
list(df)

['id',
 'age',
 'gender',
 'height',
 'weight',
 'ap_hi',
 'ap_lo',
 'cholesterol',
 'gluc',
 'smoke',
 'alco',
 'active',
 'cardio']

In [177]:
display("NULL Values", df.isnull().sum())

'NULL Values'

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [178]:
display("Description",df.describe())

'Description'

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,53.339358,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,6.759594,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,29.583562,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,48.394521,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,53.980822,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,58.430137,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,64.967123,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [179]:
df.corr()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,1.000000,0.003457,0.003502,-0.003038,-0.001830,0.003356,-0.002529,0.006106,0.002467,-0.003699,0.001210,0.003755,0.003799
age,0.003457,1.000000,-0.022811,-0.081515,0.053684,0.020764,0.017647,0.154424,0.098703,-0.047633,-0.029723,-0.009927,0.238159
gender,0.003502,-0.022811,1.000000,0.499033,0.155406,0.006005,0.015254,-0.035821,-0.020491,0.338135,0.170966,0.005866,0.008109
height,-0.003038,-0.081515,0.499033,1.000000,0.290968,0.005488,0.006150,-0.050226,-0.018595,0.187989,0.094419,-0.006570,-0.010821
weight,-0.001830,0.053684,0.155406,0.290968,1.000000,0.030702,0.043710,0.141768,0.106857,0.067780,0.067113,-0.016867,0.181660
ap_hi,0.003356,0.020764,0.006005,0.005488,0.030702,1.000000,0.016086,0.023778,0.011841,-0.000922,0.001408,-0.000033,0.054475
ap_lo,-0.002529,0.017647,0.015254,0.006150,0.043710,0.016086,1.000000,0.024019,0.010806,0.005186,0.010601,0.004780,0.065719
cholesterol,0.006106,0.154424,-0.035821,-0.050226,0.141768,0.023778,0.024019,1.000000,0.451578,0.010354,0.035760,0.009911,0.221147
gluc,0.002467,0.098703,-0.020491,-0.018595,0.106857,0.011841,0.010806,0.451578,1.000000,-0.004756,0.011246,-0.006770,0.089307
smoke,-0.003699,-0.047633,0.338135,0.187989,0.067780,-0.000922,0.005186,0.010354,-0.004756,1.000000,0.340094,0.025858,-0.015486


In [180]:
y = df[["cardio"]]
X = df.drop(["id","cardio"],axis=1)

In [126]:
# y = df[["Severity_None","Severity_Mild","Severity_Moderate","Severity_Severe"]]
# y.head(10)

In [127]:
# y.describe()

In [128]:
# X = df.drop(["Severity_None","Severity_Mild","Severity_Moderate","Severity_Severe"],axis=1)
# X.head()

In [129]:
# df.dtypes

In [130]:
# Generate our categorical variable list
# df_cat = df.dtypes[df.dtypes == "object"].index.tolist()
# df_cat

In [136]:
# # Create a OneHotEncoder instance
# enc = OneHotEncoder(sparse=False)

# # Fit and transform the OneHotEncoder using the categorical variable list
# encode_df = pd.DataFrame(enc.fit_transform(df[df_cat]))

# # Add the encoded variable names to the DataFrame
# encode_df.columns = enc.get_feature_names(df_cat)
# encode_df.head()

In [135]:
# Merge one-hot encoded features and drop the originals
# merged_df = df.merge(encode_df,left_index=True, right_index=True)
# merged_df

In [134]:
# cleaned_df = merged_df.drop("Country",axis=1)
# cleaned_df

In [133]:
# X = cleaned_df.drop(["Severity_None","Severity_Mild","Severity_Moderate","Severity_Severe"],axis=1)
# X.head()

In [132]:
# y = df[["Severity_None","Severity_Mild","Severity_Moderate","Severity_Severe"]]
# y.head(10)

In [131]:
# y = df[["Severity_Severe"]]
# y.head(10)

In [148]:
# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [183]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [185]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 36
hidden_nodes_layer2 = 8
# hidden_nodes_layer3 = 70

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 36)                432       
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 296       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 9         
Total params: 737
Trainable params: 737
Non-trainable params: 0
_________________________________________________________________


In [186]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [187]:
# Print the model summary
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 36)                432       
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 296       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 9         
Total params: 737
Trainable params: 737
Non-trainable params: 0
_________________________________________________________________


In [188]:
X_train_scaled.shape

(52500, 11)

In [189]:
# Use the training data to fit (train) the model
# @NOTE: Experiment with the number of training epochs to find the minimum iterations required to achieve a good accuracy
nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.6152 - accuracy: 0.6673
Epoch 2/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5666 - accuracy: 0.7203
Epoch 3/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5586 - accuracy: 0.7280
Epoch 4/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5574 - accuracy: 0.7275
Epoch 5/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5563 - accuracy: 0.7289
Epoch 6/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5552 - accuracy: 0.7282
Epoch 7/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5540 - accuracy: 0.7285
Epoch 8/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5528 - accuracy: 0.7293
Epoch 9/100
1641/1641 [==============================] - 2s 1ms/step - loss: 0.5522 - accuracy: 0.7281
Epoch 10/100
1641/1641 [==============================] - 2s 1ms/step - l

In [168]:
# Export our model to HDF5 file
nn.save("trained_factors.h5")

In [ ]:
import tensorflowjs as tfjs

In [138]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

In [139]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.697


In [140]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a classifier object
learning_rates = [0.01,0.05, 0.1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=80,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=8,
                                            random_state=42)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()

Learning rate:  0.01
Accuracy score (training): 0.751
Accuracy score (validation): 0.736

Learning rate:  0.05
Accuracy score (training): 0.767
Accuracy score (validation): 0.737

Learning rate:  0.1
Accuracy score (training): 0.778
Accuracy score (validation): 0.736



In [156]:
X_test

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
6670,16099,1,153,93.0,140,90,1,1,0,0,1
49567,14437,1,157,89.0,120,80,1,1,0,0,1
50796,19839,1,165,66.0,150,90,1,1,0,0,1
22310,18050,2,176,88.0,160,100,2,2,1,1,1
54037,16819,2,165,78.0,140,90,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
32138,21193,2,165,69.0,140,80,1,1,0,0,1
53648,22458,1,156,87.0,160,90,3,3,0,0,1
64554,20583,2,172,68.0,140,90,1,1,0,0,1
33812,22179,2,168,70.0,140,80,2,1,1,1,1


In [164]:
X_test_row = X_test[0:1]

In [165]:
X_test_row

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
6670,16099,1,153,93.0,140,90,1,1,0,0,1


In [167]:
nn.predict(X_test_row)

array([[0.8518666]], dtype=float32)